<a href="https://colab.research.google.com/github/namantam1/ml-ai-dnn/blob/main/Word_embedding_using_tesorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised Learning

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [ ]:
reviews = ['nice food',
        'amazing restaurant',
        'too good',
        'just loved it!',
        'will go again',
        'horrible food',
        'never go there',
        'poor service',
        'poor quality',
        'needs improvement']

sentiment = [1,1,1,1,1,0,0,0,0,0]

vocabolary_size = 100
max_length = 4
out_embedded_size = 5

df = pd.DataFrame(zip(reviews, sentiment), columns=["review", "sentiment"])
df.head()

,review,sentiment
0,nice food,1
1,amazing restaurant,1
2,too good,1
3,just loved it!,1
4,will go again,1


In [ ]:
tf.keras.preprocessing.text.one_hot(reviews[1], 10)

[2, 1]

In [ ]:
encoded_review = df.review.apply(lambda x: tf.keras.preprocessing.text.one_hot(x, vocabolary_size))
encoded_review

0        [48, 22]
1        [47, 46]
2         [36, 5]
3    [20, 35, 84]
4     [1, 54, 19]
5        [32, 22]
6    [52, 54, 55]
7        [64, 94]
8        [64, 82]
9         [11, 2]
Name: review, dtype: object

In [ ]:
padded_review = tf.keras.preprocessing.sequence.pad_sequences(encoded_review, maxlen=max_length, padding="post")
padded_review

array([[48, 22,  0,  0],
       [47, 46,  0,  0],
       [36,  5,  0,  0],
       [20, 35, 84,  0],
       [ 1, 54, 19,  0],
       [32, 22,  0,  0],
       [52, 54, 55,  0],
       [64, 94,  0,  0],
       [64, 82,  0,  0],
       [11,  2,  0,  0]], dtype=int32)

In [ ]:
model = tf.keras.models.Sequential(layers=[
    tf.keras.layers.Embedding(input_dim=vocabolary_size, output_dim=out_embedded_size, input_length=max_length)
])

model.summary()

res = model.predict(padded_review[:1])
display(res.shape, res)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 5)              500       
                                                                 
Total params: 500
Trainable params: 500
Non-trainable params: 0
_________________________________________________________________
1/1 [==============================] - 0s 395ms/step


(1, 4, 5)

array([[[ 0.04483915,  0.04730549,  0.03211022, -0.03175589,
         -0.03915942],
        [-0.02853294,  0.02670611, -0.04709234,  0.04388363,
         -0.03390338],
        [ 0.0480032 ,  0.01492996, -0.00394572, -0.01552893,
          0.03739699],
        [ 0.0480032 ,  0.01492996, -0.00394572, -0.01552893,
          0.03739699]]], dtype=float32)

In [ ]:
model = tf.keras.models.Sequential(layers=[
    tf.keras.layers.Embedding(input_dim=vocabolary_size, output_dim=out_embedded_size, 
                              input_length=max_length, name="embedding"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=100, activation="relu"),
    tf.keras.layers.Dense(units=50, activation="relu"),
    tf.keras.layers.Dense(units=1, activation="sigmoid"),
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 5)              500       
                                                                 
 flatten_1 (Flatten)         (None, 20)                0         
                                                                 
 dense_3 (Dense)             (None, 100)               2100      
                                                                 
 dense_4 (Dense)             (None, 50)                5050      
                                                                 
 dense_5 (Dense)             (None, 1)                 51        
                                                                 
Total params: 7,701
Trainable params: 7,701
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(padded_review, df.sentiment, epochs=1000)

Epoch 1/1000
1/1 [==============================] - 1s 546ms/step - loss: 0.6933 - accuracy: 0.5000
Epoch 2/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.6910 - accuracy: 0.7000
Epoch 3/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.6891 - accuracy: 1.0000
Epoch 4/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.6872 - accuracy: 1.0000
Epoch 5/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.6853 - accuracy: 1.0000
Epoch 6/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.6836 - accuracy: 1.0000
Epoch 7/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.6817 - accuracy: 1.0000
Epoch 8/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.6797 - accuracy: 1.0000
Epoch 9/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.6776 - accuracy: 1.0000
Epoch 10/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.6753 - accuracy: 1.0000
Epoch 11/1000
1/

In [ ]:
predicted = model.predict(padded_review)

display(df.sentiment.to_numpy(), predicted.reshape(-1))

1/1 [==============================] - 0s 57ms/step


array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])

array([9.9996042e-01, 9.9998915e-01, 9.9999154e-01, 9.9999666e-01,
       9.9997663e-01, 9.1626185e-05, 7.6990378e-05, 3.1882966e-05,
       2.6149066e-05, 4.3791046e-05], dtype=float32)

In [ ]:
embedded_layer = model.get_layer("embedding")
weights = np.array(embedded_layer.get_weights()[0])

display(weights.shape, weights[0])

(100, 5)

array([ 0.14726722,  0.14564395, -0.15086071,  0.1396672 , -0.13746569],
      dtype=float32)